In [1]:
# !pip install pyomo==6.4.1
# !apt install glpk-utils
# !pip install "git+https://github.com/sjpfenninger/sen1511.git#egg=sen1511utils&subdirectory=sen1511utils"

In [2]:
import pyomo.environ as pyo

from sen1511utils import summarise_results

## Stochastic Programming

In the “Text to study – Robust optimization example” we looked at the problem of an electricity consumer facing both uncertain electricity price for next week (24*7= 168hrs) and addressed this decision problem with robust optimisation.
We now want to approach the same problem with stochastic programming. To do so we consider additionally that not only price, but also demand is uncertain. Both remain constant throughout the week. 
Scenario data for demand and price are provided in the table:

|Scenario data for the consumer||||
|:---|---:|---:|---:|
| Scenario # | Probability (per unit) | Demand [MW]| Price [€/MWh]|
|1|0.2|110|50|
|2|0.6|100|46|
|3|0.2|80 |44|

The rest of the problem remains the same: the consumer has the possibility of buying up to 90 MW at €45/MWh throughout next week, by signing a bilateral contract before next week, i.e., before knowing the actual demand and pool price it has to face.

The decision-making problem of this consumer can be formulated as a stochastic programming problem: the consumer has to decide how much to buy from the contract PC, and to decide his pool purchases for each of the three considered demand/price realizations (scenarios) P1, P2, and P3.

The objective function is the expected cost faced by the consumer to supply its uncertain demand.


# Q.3)
Solve the problem and provide the objective function value and the optimal decisions.


Note: Tasks 1) through 2.c) are solved on paper.

In [8]:
model = pyo.ConcreteModel(name = "Stochastic Model")
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

model.Pc = pyo.Var(domain=pyo.NonNegativeReals)
model.P1 = pyo.Var(domain=pyo.NonNegativeReals)
model.P2 = pyo.Var(domain=pyo.NonNegativeReals)
model.P3 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

model.profit = pyo.Objective(
    expr = (168 * (45 * model.Pc + 0.2 * 50 * model.P1 + 0.6 * 46 * model.P2 + 0.2 * 44 * model.P3)),
    sense = pyo.minimize,
)

##
# 3. Constraints
##

model.demand1 = pyo.Constraint(expr = model.Pc + model.P1 >=  110)
model.demand2= pyo.Constraint(expr = model.Pc + model.P2 >=  100)
model.demand3 = pyo.Constraint(expr = model.Pc + model.P3 >=  80)

model.LowerLimitc = pyo.Constraint(expr = model.Pc >= 0)
model.LowerLimit1 = pyo.Constraint(expr = model.P1 >= 0)
model.LowerLimit2 = pyo.Constraint(expr = model.P2 >= 0)
model.LowerLimit3 = pyo.Constraint(expr = model.P3 >= 0)

model.UpperLimit2 = pyo.Constraint(expr = model.Pc <= 90)

# Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(model)


{'Problem': [{'Name': 'unknown', 'Lower bound': 747936.0, 'Upper bound': 747936.0, 'Number of objectives': 1, 'Number of constraints': 9, 'Number of variables': 5, 'Number of nonzeros': 12, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.032181739807128906}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [9]:
summarise_results(model)